<a href="https://colab.research.google.com/github/hamk3010/DeepLearning/blob/master/Linear%20Regression%20with%20TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

# Fetch the housing dataset
housing = fetch_california_housing()
m,n = housing.data.shape

# Add a bias input feature (x0 = 1) to all training instances
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

In [0]:
#  Construction phase
# Create two TensorFlow constant nodes, X and y to hold this data and the targets
X = tf.constant(housing_data_plus_bias, dtype= tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1,1), dtype= tf.float32, name = "y")
XT = tf.transpose(X)
theta = tf.matmul( tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

In [0]:
# Execution phase

with tf.Session() as sess:
    theta_value = theta.eval()

In [4]:
print(theta_value)

[[-3.71037292e+01]
 [ 4.36282694e-01]
 [ 9.40542948e-03]
 [-1.06901854e-01]
 [ 6.43611908e-01]
 [-4.06625077e-06]
 [-3.78273334e-03]
 [-4.23094332e-01]
 [-4.36462164e-01]]
